In [1]:
from transformers import PreTrainedTokenizerFast,GPT2LMHeadModel
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [2]:
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re

In [3]:
data = pd.read_csv('Textranked.csv')
DEVICE =  torch.device("cuda" if torch.cuda.is_available() else "cpu")
data.dropna(inplace=True)

In [4]:
data

,Question,Answer
0,정말 어려운 상황에서 너무나도 힘들어서 질문을 드립니다. 아버지가 제 명의로 사업자...,차량의 경우 현재 점유자인 아버지를 피고로 하여 차량 인도 청구소송 및 차량 점유 ...
1,년 전에 대포 통장이로 인해 피해를 봐서 재판 결과가 불기소 유통 지을 받았거든요....,소장 내용에 대한 검토가 필요하므로 소장을 지참하고 상담을 받으시기 바랍니다
2,제 계좌로 입금을 했다던가 그런부분 전혀없이 그 형의 계좌로 그 형의 공인인증서를 ...,별도의 구체적 상담이 필요하나 실제 채무가 없음에도 공증을 가지고 채권자가 강제집행...
3,채권시효년 소멸로 본다면 어떤 대응이 가능할지 여쭤봅니다 지인에게 연대보증 해줬는데...,별도의 구체적 상담이 필요하나 이행 권고 결정은 기판력의 제한이 없으므로 소멸시효 ...
4,그 와중에 피의자와 어떻게 연락이 닿게 되어서 갚겠다는 의사를 보였고 정말 적은 ...,가해자가 위 사건으로 구속 기소되었다면 가해자를 피고로 하여 민사소송 및 강제집행으...
...,...,...
18160,제가 작년 월쯤 승소 판결 받은 후에 압류 절차를 진행을 해야 하는데 혼자 스스로 ...,여쭤보신 것은 통장 압류 절차인 것으로 보입니다 통장 압류의 법률적 용어는 예금채...
18161,민사재판 패소 후 현재 등본은 부모님댁으로 되어있습니다,민사소송 패소 후 부모님 집에 압류가 들어올 수 있는지와 관련하여 질문하셨습니다.민...
18162,이에 원고인에게 원룸계약서 제출을 요구합니다 라고 썼거든요.거기보니 원고측에서 이미...,따라서 각 월 차임 채권은 년에 발생한 경우 년이 경과한 년 사이에 이미 시효...
18163,오늘 피고 측에서 답변서를 받고 변론 기일이 정해졌는데 이제 보니 제가 작성한 소...,민사소송 소장에 소송비용 청구를 빠드린 경우와 관련하여 질 무하셨습니다. 민사소송에...


In [5]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<mask>'
SENT = '<sent>'
PAD = '<pad>'

In [6]:
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=1024):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = tokenizer

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Question"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["Answer"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [7]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [8]:
train_set = ChatbotDataset(data, max_len=1024)
#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=1, num_workers=0, pin_memory=True,shuffle=True, collate_fn=collate_batch)

In [11]:
print("start")
for batch_idx, samples in enumerate(train_dataloader):
    token_ids, mask, label = samples
    print("token_ids ====> ", token_ids)
    print("mask =====> ", mask)
    print("label =====> ", label)
print("end")

start
token_ids ====>  tensor([[    2,  9490, 18866,  ...,     3,     3,     3],
        [    2,  9306, 16069,  ...,     3,     3,     3],
        [    2,  9539,  9025,  ...,     3,     3,     3],
        [    2, 11059, 13588,  ...,     3,     3,     3]])
mask =====>  tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
label =====>  tensor([[6, 6, 6,  ..., 3, 3, 3],
        [6, 6, 6,  ..., 3, 3, 3],
        [6, 6, 6,  ..., 3, 3, 3],
        [6, 6, 6,  ..., 3, 3, 3]])
token_ids ====>  tensor([[    2,  9252,  7220,  ...,     3,     3,     3],
        [    2,  9265,  7162,  ...,     3,     3,     3],
        [    2, 10795,  9359,  ...,     3,     3,     3],
        [    2, 12764, 24240,  ...,     3,     3,     3]])
mask =====>  tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
label =====>  tensor([[6, 6, 6,  ..., 3, 3,

In [9]:
model.to(DEVICE)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [10]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 10
Sneg = -1e18

In [11]:
from tqdm import tqdm
model.train()
print ("start")
for epoch in tqdm(range(epoch)):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        token_ids = token_ids.cuda(non_blocking=True)
        mask = mask.cuda(non_blocking=True)
        label = label.cuda(non_blocking=True)
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
        if batch_idx % 10 == 9:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {batch_idx + 1:5d}] loss: {avg_loss / 2000:.3f}')
            avg_loss = 0.0
print ("end")

start


  0%|          | 0/10 [00:00<?, ?it/s]C:\Users\lucky\AppData\Local\Temp\ipykernel_26588\2495599640.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)


[1,    10] loss: 0.071
[1,    20] loss: 0.090
[1,    30] loss: 0.044
[1,    40] loss: 0.068
[1,    50] loss: 0.117
[1,    60] loss: 0.789
[1,    70] loss: 0.170
[1,    80] loss: 0.072
[1,    90] loss: 0.098
[1,   100] loss: 0.108
[1,   110] loss: 0.228
[1,   120] loss: 0.089
[1,   130] loss: 0.322
[1,   140] loss: 0.023
[1,   150] loss: 0.202
[1,   160] loss: 0.023
[1,   170] loss: 0.032
[1,   180] loss: 0.039
[1,   190] loss: 0.500
[1,   200] loss: 0.131
[1,   210] loss: 0.176
[1,   220] loss: 0.084
[1,   230] loss: 0.068
[1,   240] loss: 0.067
[1,   250] loss: 0.393
[1,   260] loss: 0.159
[1,   270] loss: 0.155
[1,   280] loss: 0.024
[1,   290] loss: 0.160
[1,   300] loss: 0.228
[1,   310] loss: 0.209
[1,   320] loss: 0.060
[1,   330] loss: 0.159
[1,   340] loss: 0.218
[1,   350] loss: 0.026
[1,   360] loss: 0.013
[1,   370] loss: 0.061
[1,   380] loss: 0.134
[1,   390] loss: 0.219
[1,   400] loss: 0.260
[1,   410] loss: 0.028
[1,   420] loss: 0.237
[1,   430] loss: 0.049
[1,   440] 

 10%|█         | 1/10 [1:35:54<14:23:14, 5754.90s/it]

[2,    10] loss: 0.201
[2,    20] loss: 0.046
[2,    30] loss: 0.132
[2,    40] loss: 0.055
[2,    50] loss: 0.049
[2,    60] loss: 0.423
[2,    70] loss: 0.201
[2,    80] loss: 0.072
[2,    90] loss: 0.342
[2,   100] loss: 0.104
[2,   110] loss: 0.142
[2,   120] loss: 0.154
[2,   130] loss: 0.023
[2,   140] loss: 0.134
[2,   150] loss: 0.238
[2,   160] loss: 0.107
[2,   170] loss: 0.304
[2,   180] loss: 0.194
[2,   190] loss: 0.055
[2,   200] loss: 0.019
[2,   210] loss: 0.025
[2,   220] loss: 0.193
[2,   230] loss: 0.116
[2,   240] loss: 0.218
[2,   250] loss: 0.051
[2,   260] loss: 0.028
[2,   270] loss: 0.022
[2,   280] loss: 0.039
[2,   290] loss: 0.092
[2,   300] loss: 0.024
[2,   310] loss: 0.095
[2,   320] loss: 0.073
[2,   330] loss: 0.088
[2,   340] loss: 0.260
[2,   350] loss: 0.023
[2,   360] loss: 0.029
[2,   370] loss: 0.201
[2,   380] loss: 0.226
[2,   390] loss: 0.079
[2,   400] loss: 0.248
[2,   410] loss: 0.089
[2,   420] loss: 0.094
[2,   430] loss: 0.044
[2,   440] 

...

[9,    10] loss: 0.236
[9,    20] loss: 0.030
[9,    30] loss: 0.078
[9,    40] loss: 0.116
[9,    50] loss: 0.034
[9,    60] loss: 0.019
[9,    70] loss: 0.158
[9,    80] loss: 0.145
[9,    90] loss: 0.111
[9,   100] loss: 0.137
[9,   110] loss: 0.064
[9,   120] loss: 0.391
[9,   130] loss: 0.087
[9,   140] loss: 0.020
[9,   150] loss: 0.611
[9,   160] loss: 0.118
[9,   170] loss: 0.023
[9,   180] loss: 0.321
[9,   190] loss: 0.050
[9,   200] loss: 0.032
[9,   210] loss: 0.127
[9,   220] loss: 0.008
[9,   230] loss: 0.028
[9,   240] loss: 0.039
[9,   250] loss: 0.029
[9,   260] loss: 0.015
[9,   270] loss: 0.422
[9,   280] loss: 0.200
[9,   290] loss: 0.457
[9,   300] loss: 0.168
[9,   310] loss: 0.180
[9,   320] loss: 0.217
[9,   330] loss: 0.028
[9,   340] loss: 0.174
[9,   350] loss: 0.033
[9,   360] loss: 0.023
[9,   370] loss: 0.145
[9,   380] loss: 0.303
[9,   390] loss: 0.061
[9,   400] loss: 0.094
[9,   410] loss: 0.193
[9,   420] loss: 0.023
[9,   430] loss: 0.342
[9,   440] 

 90%|█████████ | 9/10 [14:21:43<1:35:47, 5747.30s/it]

[10,    10] loss: 0.079
[10,    20] loss: 0.217
[10,    30] loss: 0.068
[10,    40] loss: 0.013
[10,    50] loss: 0.259
[10,    60] loss: 0.247
[10,    70] loss: 0.208
[10,    80] loss: 0.044
[10,    90] loss: 0.174
[10,   100] loss: 0.121
[10,   110] loss: 0.104
[10,   120] loss: 0.023
[10,   130] loss: 0.023
[10,   140] loss: 0.034
[10,   150] loss: 0.100
[10,   160] loss: 0.033
[10,   170] loss: 0.141
[10,   180] loss: 0.236
[10,   190] loss: 0.200
[10,   200] loss: 0.153
[10,   210] loss: 0.059
[10,   220] loss: 0.145
[10,   230] loss: 0.163
[10,   240] loss: 0.071
[10,   250] loss: 0.008
[10,   260] loss: 0.099
[10,   270] loss: 0.247
[10,   280] loss: 0.226
[10,   290] loss: 0.031
[10,   300] loss: 0.041
[10,   310] loss: 0.689
[10,   320] loss: 0.153
[10,   330] loss: 0.303
[10,   340] loss: 0.200
[10,   350] loss: 0.035
[10,   360] loss: 0.226
[10,   370] loss: 0.096
[10,   380] loss: 0.272
[10,   390] loss: 0.110
[10,   400] loss: 0.342
[10,   410] loss: 0.032
[10,   420] loss

100%|██████████| 10/10 [15:57:32<00:00, 5745.25s/it] 

end


In [12]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer.encode(Q_TKN + q + A_TKN + a)).unsqueeze(dim=0).to(DEVICE)
            pred = model(input_ids)
            pred = pred.logits.to('cpu')
            gen = tokenizer.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))

Chatbot > 질문자님의 자동차 대국민 정보가 불법행위에 기한 손해배상청구권의 단기소멸시효의 완성으로 자동차 소유권이전등기의 말소등기청구권의 소멸시효는 년입니다  그러나 질문자님의 경우 자동차 소유자가 질문자님의 명의로 된 차량을 아버지가 소유하였다고 한다면 시효는 완성되지 않았을 가능성이 있습니다
Chatbot > 우선 결론부터 말씀드리면  항고장 제출은 관할 법원이 원칙이지만 금전 청구라면 채권자는 주소지 관할법원에 손해배상을 청구할 수 있습니다


In [14]:
import os

MODEL_PATH = './models'
model_name = 'koGPT2_finetuned'

save_path = os.path.join(MODEL_PATH, model_name)

model.save_pretrained(save_path)

In [15]:
tokenizer.save_pretrained('./models/koGPT2_tokenizer')

('./models/koGPT2_tokenizer\\tokenizer_config.json',
 './models/koGPT2_tokenizer\\special_tokens_map.json',
 './models/koGPT2_tokenizer\\tokenizer.json')

In [16]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("./models/koGPT2_tokenizer",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')
model = GPT2LMHeadModel.from_pretrained('./models/koGPT2_finetuned')

In [14]:
data['Question'][0]

'정말 어려운 상황에서 너무나도 힘들어서 질문을 드립니다. 부디 자세한 설명해주시면 감사하겠습니다. 아버지가 제 명의로 사업자를 내서 사업을 했으나 늘어나는 것은 빛뿐이었습니다. 더 이상은 방관할 수가 없어 아버지에게 제 명의로 해드린 것을 모두 정지시키려고 하는 상황입니다. 또 어머니 명의의 집을 팔아서 빚을 메꾸려고 합니다. 첫 번째 질문입니다. 어머니 명의로 된 집 아버지 어머니 함께 거주를 매도하려고 하는데 그 과정에서 아버지가 방해를 하고 가구 이동을 못 시키게 하고 그러면 어떤 방법을 취할 수 있을까요 두 번째 질문입니다. 제 명의로 된 차량을 아버지가 사용하고 계셨는데 제가 도로 갖고 와도 될까요. 아니면 어떤 방법이 있을까요 답변해 주시면 정말 감사드리겠습니다'

In [15]:
data['Answer'][0]

'차량의 경우 현재 점유자인 아버지를 피고로 하여 차량 인도 청구소송 및 차량 점유 이전금지 가처분으로 차량을 조기에 회수할 수 있습니다'